In [7]:
import numpy as np 
max_steps = 10
tolerance = 0.1
# Define the environment
class Environment:
    def __init__(self, size, robot_positions, goals):
        self.size = size
        self.robot_positions = robot_positions
        self.goals = goals

# Define the belief states
class BeliefState:
    def __init__(self, positions, velocities, goals):
        self.positions = positions
        self.velocities = velocities
        self.goals = goals

# Define the cost function
def cost_function(position, goal, other_robot_position):
    direct_distance = np.linalg.norm(position - goal)
    arc_distance = np.linalg.norm(position - other_robot_position)
    return direct_distance - arc_distance

# Prediction step
def predict_next_state(belief_state, action):
    # Update positions based on action
    new_positions = belief_state.positions + action
    return BeliefState(new_positions, belief_state.velocities, belief_state.goals)

# Update step
def update_belief(observation, belief_state):
    # Update the belief based on the observation
    new_positions = observation
    return BeliefState(new_positions, belief_state.velocities, belief_state.goals)

# Policy evaluation
def evaluate_policy(belief_state, actions):
    min_expected_free_energy = float('inf')
    best_action = None
    for action in actions:
        predicted_state = predict_next_state(belief_state, action)
        cost = cost_function(predicted_state.positions[0], belief_state.goals[0], belief_state.positions[1])
        if cost < min_expected_free_energy:
            min_expected_free_energy = cost
            best_action = action
    return best_action

# Action selection and execution
def execute_policy(environment, belief_state, actions):
    for step in range(max_steps):
        best_action = evaluate_policy(belief_state, actions)
        new_positions = environment.robot_positions + best_action
        environment.robot_positions = new_positions
        belief_state = update_belief(new_positions, belief_state)
        # Check for convergence
        if np.linalg.norm(environment.robot_positions - environment.goals) < tolerance:
            break
    return environment.robot_positions

# Example usage
env = Environment(size=10, robot_positions=np.array([[0, 0], [10, 10]]), goals=np.array([[5, 5], [5, 0]]))
belief_state = BeliefState(positions=env.robot_positions, velocities=np.array([[0, 0], [0, 0]]), goals=env.goals)
actions = [np.array([0.1, 0.1]), np.array([0.1, -0.1]), np.array([-0.1, 0.1]), np.array([-0.1, -0.1])]

final_positions = execute_policy(env, belief_state, actions)
print("Final positions of robots:", final_positions)


Final positions of robots: [[ 0.2  0.2]
 [10.2 10.2]]
